In [1]:
import numpy as np
import os
import time
import datetime as dt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms, datasets
from torch.autograd import Variable
import torchvision

In [2]:
# Send the model to the GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [101]:
# Define transforms to be applied to the image data
transform = transforms.Compose([
    transforms.Resize(64),
    #transforms.CenterCrop(100),
    #transforms.RandomHorizontalFlip(),
    #transforms.RandomVerticalFlip(),
    #transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [102]:
# Load the dataset
train_dataset = datasets.ImageFolder("D:/tay/Data/Data_crop/Data_split/classes_image/image_train/", transform=transform)

# Define the data loader
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)

# Define the label names
label_names = train_dataset.classes

In [103]:
# Load the dataset
val_dataset = datasets.ImageFolder("D:/tay/Data/Data_crop/Data_split/classes_image/image_test/", transform=transform)

# Define the data loader
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

# Define the label names
label_names = val_dataset.classes

In [22]:
print(label_names)

['hand_A', 'hand_A2', 'hand_B', 'hand_C', 'hand_D', 'hand_D2', 'hand_E', 'hand_G', 'hand_H', 'hand_I', 'hand_K', 'hand_L', 'hand_M', 'hand_N', 'hand_O', 'hand_O3', 'hand_P', 'hand_Q', 'hand_R', 'hand_S', 'hand_T', 'hand_U', 'hand_V', 'hand_X', 'hand_Y']


In [111]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU()
        self.pool3 = nn.MaxPool2d(2)
        self.fc = nn.Linear(128 * 8 * 8, 25)  # fully-connected layer for classification

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = x.view(-1, 128 * 8 * 8)  # flatten the output of the last convolutional layer
        x = self.fc(x)
        return x

In [112]:
model = ConvNet().to(device)
print(model)

ConvNet(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): ReLU()
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): ReLU()
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): ReLU()
  (pool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc): Linear(in_features=8192, out_features=25, bias=True)
)


In [113]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
num_epochs = 1
start_time = time.time()
# Loop over the dataset and train the model
model.train()
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        # Zero the gradients
        optimizer.zero_grad()
        
        #send data to cuda
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Print training statistics
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Train time: {:.2f}'
                  .format(epoch+1, num_epochs, i+1, len(train_loader), loss.item(), (time.time()-start_time)/60))
'''
    # Evaluate the model on the validation dataset
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy on validation set: {:.2f}%'.format(100 * correct / total))
'''


Epoch [1/1], Step [100/274360], Loss: 27.5935, Train time: 0.04
Epoch [1/1], Step [200/274360], Loss: 2.8099, Train time: 0.08
Epoch [1/1], Step [300/274360], Loss: 3.9319, Train time: 0.12
Epoch [1/1], Step [400/274360], Loss: 1.1444, Train time: 0.16
Epoch [1/1], Step [500/274360], Loss: 4.1925, Train time: 0.20
Epoch [1/1], Step [600/274360], Loss: 1.2972, Train time: 0.28
Epoch [1/1], Step [700/274360], Loss: 3.9364, Train time: 0.32
Epoch [1/1], Step [800/274360], Loss: 0.0151, Train time: 0.37
Epoch [1/1], Step [900/274360], Loss: 9.6515, Train time: 0.46
Epoch [1/1], Step [1000/274360], Loss: 3.1977, Train time: 0.50
Epoch [1/1], Step [1100/274360], Loss: 3.1546, Train time: 0.59
Epoch [1/1], Step [1200/274360], Loss: 2.0256, Train time: 0.63
Epoch [1/1], Step [1300/274360], Loss: 2.8301, Train time: 0.68
Epoch [1/1], Step [1400/274360], Loss: 1.2383, Train time: 0.75
Epoch [1/1], Step [1500/274360], Loss: 0.8676, Train time: 0.80
Epoch [1/1], Step [1600/274360], Loss: 5.1049, T

"\n    # Evaluate the model on the validation dataset\n    correct = 0\n    total = 0\n    with torch.no_grad():\n        for inputs, labels in val_loader:\n            outputs = model(inputs)\n            _, predicted = torch.max(outputs.data, 1)\n            total += labels.size(0)\n            correct += (predicted == labels).sum().item()\n\n    print('Accuracy on validation set: {:.2f}%'.format(100 * correct / total))\n"

In [114]:
# Evaluate the model on the validation dataset
for epoch in range(num_epochs):
    start_time = time.time()
    correct = 0
    total = 0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            # Print training statistics
            if (i+1) % 100 == 0:
                print('Epoch [{}/{}], Step [{}/{}], Accuracy: {:.2f}, Test time: {:.2f}'
                      .format(epoch+1, num_epochs, i+1, len(val_loader), 100 * correct / total, (time.time()-start_time)/60))

    print('Accuracy on test set: {:.2f}%'.format(100 * correct / total))

Epoch [1/1], Step [100/90866], Accuracy: 76.00, Test time: 0.05
Epoch [1/1], Step [200/90866], Accuracy: 80.00, Test time: 0.10
Epoch [1/1], Step [300/90866], Accuracy: 80.33, Test time: 0.26
Epoch [1/1], Step [400/90866], Accuracy: 80.25, Test time: 0.34
Epoch [1/1], Step [500/90866], Accuracy: 80.20, Test time: 0.42
Epoch [1/1], Step [600/90866], Accuracy: 80.00, Test time: 0.50
Epoch [1/1], Step [700/90866], Accuracy: 80.29, Test time: 0.56
Epoch [1/1], Step [800/90866], Accuracy: 80.38, Test time: 0.61
Epoch [1/1], Step [900/90866], Accuracy: 80.78, Test time: 0.66
Epoch [1/1], Step [1000/90866], Accuracy: 80.70, Test time: 0.72
Epoch [1/1], Step [1100/90866], Accuracy: 80.73, Test time: 0.77
Epoch [1/1], Step [1200/90866], Accuracy: 80.83, Test time: 0.82
Epoch [1/1], Step [1300/90866], Accuracy: 81.23, Test time: 0.89
Epoch [1/1], Step [1400/90866], Accuracy: 80.86, Test time: 0.94
Epoch [1/1], Step [1500/90866], Accuracy: 81.00, Test time: 1.00
Epoch [1/1], Step [1600/90866], Ac